In [ ]:
import os
import zipfile
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess_input
from tensorflow.keras import layers, models, optimizers
from sklearn.utils import class_weight
from google.colab import drive

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)



drive.mount('/content/drive')
ZIP_PATH = '/content/drive/MyDrive/processed_dataset.zip'
EXTRACT_TO = '/content/dataset'
IMG_SIZE = (380, 380)
BATCH_SIZE = 32
EPOCHS_PHASE1 = 15
EPOCHS_PHASE2 = 65
LEARNING_RATE_PHASE1 = 0.0001
LEARNING_RATE_PHASE2 = 0.00001

if not os.path.exists(EXTRACT_TO):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_TO)

base_dir = EXTRACT_TO
if 'processed_dataset' in os.listdir(base_dir):
    base_dir = os.path.join(base_dir, 'processed_dataset')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'test')


train_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess_input,
    rotation_range=40, horizontal_flip=True, zoom_range=0.3, shear_range=0.3,
    brightness_range=[0.5, 1.5], fill_mode='nearest'
)
val_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

y_train = train_generator.classes
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)
class_weights = dict(enumerate(class_weights_array))

df_index = 3; mel_index = 4; bcc_index = 1; bkl_index = 2; vasc_index = 6; akiec_index = 0


class_weights[bkl_index] = class_weights[bkl_index] * 0.30
class_weights[df_index] = class_weights[df_index] * 0.75


critical_indices = [mel_index, bcc_index, akiec_index, vasc_index]
for index in critical_indices:
    class_weights[index] = class_weights[index] * 2.00



print("ResNet50V2 is downloading...")

base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.000001)
]


base_model.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE_PHASE1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n--- STEP 1: Custom Head Training ({EPOCHS_PHASE1} Tur) ---")
history_phase1 = model.fit(
    train_generator, epochs=EPOCHS_PHASE1, validation_data=validation_generator,
    class_weight=class_weights, callbacks=[callbacks[1]]
)


base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.70)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE_PHASE2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n--- STEP 2: Fine-Tuning ({EPOCHS_PHASE2} Tur) ---")
history_phase2 = model.fit(
    train_generator, epochs=EPOCHS_PHASE1 + EPOCHS_PHASE2,
    initial_epoch=history_phase1.epoch[-1] + 1, validation_data=validation_generator,
    class_weight=class_weights, callbacks=callbacks
)


model.save_weights('final_resnet_oversampled_weights.weights.h5')
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)
print("\n COMPLETED!")

In [ ]:
base_model.trainable = True


fine_tune_at = int(len(base_model.layers) * 0.70)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

LEARNING_RATE_PHASE2 = 0.00001

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE_PHASE2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model is ready for fine tuning.")

In [ ]:
EPOCHS_PHASE1 = 15
EPOCHS_PHASE2 = 65
total_epochs = EPOCHS_PHASE1 + EPOCHS_PHASE2 
initial_epoch_start = 5 + 1 

print(f"\n--- STEP 2: Fine-Tuning ({EPOCHS_PHASE2} Tur) ---")
print(f"Eğitim Epoch {initial_epoch_start}'dan itibaren devam ediyor (Toplam {total_epochs} tura kadar).")


history_phase2 = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=initial_epoch_start, 
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks
)


model.save_weights('final_resnet_oversampled_weights.weights.h5')
print("\nFine tuning is completed.")

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense # Diğer katmanlar da olabilir

# 1. Model Mimarisi: (Bu sadece bir varsayımdır. Kendi ResNet mimarinizi kullanmalısınız!)
# Bu örnek, bir ResNet50'nin üstüne eklenmiş basit bir sınıflandırma başlığıdır.
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

# Kaç sınıfınız olduğunu buraya yazın (Örnek: 7 sınıf)
num_classes = 7

# ResNet50'yi yüklüyoruz (hazır ağırlıklar olmadan)
base_model = ResNet50(weights=None, include_top=False, input_shape=(380, 380, 3)) 
# 'boyut' yerine kendi görüntü boyutunuzu (örn: 224) yazın.

# Modelin üstüne kendi sınıflandırma başlığınızı ekleyin (Önemli!)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x) # Çıkış katmanı

model = Model(inputs=base_model.input, outputs=predictions)

# 2. Ağırlıkları Yükleme
weights_path = "C:/Users/USER/Desktop/projects/skin-ai-app/backend/final_resnet_oversampled_weights.weights.h5"
try:
    # `by_name=True` kullanmak, katman adları eşleştiği sürece daha güvenlidir.
    model.load_weights(weights_path, by_name=True) 
    print("Ağırlıklar başarıyla yüklendi.")
except Exception as e:
    print(f"Ağırlık yüklenirken hata oluştu: {e}")
    # Eğer bu adımda hata alırsanız, mimariniz (model) kaydedilen ağırlıklarla eşleşmiyor demektir.
    
# Modeli tahmin yapmaya hazır hale getirmek için derlemeye gerek yok, ancak 
# değerlendirme yapmak isterseniz gereklidir.
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Ağırlık yüklenirken hata oluştu: `by_name` only supports loading legacy '.h5' or '.hdf5' files. Received: C:/Users/USER/Desktop/projects/skin-ai-app/backend/final_resnet_oversampled_weights.weights.h5


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Test verilerinizin bulunduğu ana dizinin yolu
test_data_dir = "C:/Users/USER/Desktop/projects/skin-model/processed_dataset/test" # Lütfen bu yolu kendi test klasörünüzün yoluyla değiştirin.
img_height, img_width = 380, 380 # Model mimarinizle aynı olmalı

test_datagen = ImageDataGenerator(rescale=1./255) # Sadece yeniden ölçekleme (rescale)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1, # Tek tek tahmin yapmak en iyisidir
    class_mode='categorical',
    shuffle=False # SIRALAMANIN KORUNMASI İÇİN ÇOK ÖNEMLİ!
)

Found 2003 images belonging to 7 classes.


In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Tahminleri yap
Y_pred = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)

# Gerçek (true) ve Tahmin Edilen (predicted) etiketleri al
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Sınıf İsimleri (Modelin sınıf indeksleriyle eşleşmeli)
class_labels = list(test_generator.class_indices.keys())

# Confusion Matrix'i hesapla
cm = confusion_matrix(y_true, y_pred)

# Confusion Matrix'i görselleştir
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('Gerçek Etiket (True Label)')
plt.xlabel('Tahmin Edilen Etiket (Predicted Label)')
plt.show() # 

# Ek olarak: Sınıflandırma Raporu (Precision, Recall, F1-Score)
print("Sınıflandırma Raporu:")
print(classification_report(y_true, y_pred, target_names=class_labels))

ModuleNotFoundError: No module named 'sklearn'